In [2]:
import torch
import warnings
import numpy as np
import pickle
import torch_geometric.transforms as T
from HealthAwareGNN import HeteroGAT, HealthAwareCriterion
from utils import EarlyStopping, user_based_split, train, test, format_metrics, calculate_user_health_history
import os
import time
import optuna

os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = 'cpu'
print(f"Device: '{device}'")
warnings.filterwarnings('ignore')

MAX_EPOCHS = 200
PATIENCE = 50

Device: 'cuda'
Device: 'cuda'


In [3]:
processed_data_path = '../data/processed_data/processed_data_GNN.pkl'
with open(processed_data_path, 'rb') as f:
    data = pickle.load(f)
print("Processed data loaded successfully.")

for edge_type in data.edge_types:
    if hasattr(data[edge_type], 'edge_index'):
        data[edge_type].edge_index = data[edge_type].edge_index.long()

Processed data loaded successfully.


In [ ]:
def objective(trial):
    HIDDEN_CHANNELS = trial.suggest_int('HIDDEN_CHANNELS', 16, 128)  # 범위 축소
    LEARNING_RATE = trial.suggest_float('LEARNING_RATE', 1e-5, 1e-2, log=True)
    WEIGHT_DECAY = trial.suggest_float('WEIGHT_DECAY', 1e-6, 1e-3, log=True)
    DROPOUT = trial.suggest_float('DROPOUT', 0.1, 0.5)
    LAMBDA_HEALTH = trial.suggest_float('LAMBDA_HEALTH', 0.1, 1.0)
    
    print(f"\n평가 중: hidden_channels={HIDDEN_CHANNELS}, lr={LEARNING_RATE:.6f}")
    
    train_data, val_data, _ = user_based_split(data, train_ratio=0.7, val_ratio=0.1, test_ratio=0.2, neg_ratio=1.0, seed=42)
    
    orig_edge_index = data['user', 'healthness', 'food'].edge_index
    orig_health_scores = data['user', 'healthness', 'food'].edge_attr
    health_dict = {(u.item(), f.item()): s.item() for u, f, s in zip(orig_edge_index[0], orig_edge_index[1], orig_health_scores)}
    
    user_health_history = calculate_user_health_history(train_data, health_dict)
    model = HeteroGAT(
        hidden_channels=HIDDEN_CHANNELS, 
        out_channels=HIDDEN_CHANNELS, 
        metadata=train_data.metadata(),
        dropout=DROPOUT
    ).to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, 
        max_lr=LEARNING_RATE * 10,
        total_steps=MAX_EPOCHS,
        pct_start=0.3,
        div_factor=10.0,
        final_div_factor=100.0
    )    
    criterion = HealthAwareCriterion(lambda_health=LAMBDA_HEALTH)
    
    # 학습 설정
    best_val_auc = 0
    best_model_state = None
    early_stopping = EarlyStopping(patience=PATIENCE, delta=0.001, higher_is_better=True)

    for epoch in range(1, MAX_EPOCHS + 1):
        model.train()
        train_loss = train(model, optimizer, train_data, health_dict, criterion, user_health_history)
    
        model.eval()
        val_metrics = test(model, val_data, health_dict, criterion, user_health_history=user_health_history)
        val_metrics_dict = format_metrics(val_metrics)
        
        if epoch == 1 or epoch % 20 == 0:
            print(f"Epoch {epoch:03d}: Val AUC = {val_metrics_dict['auc']:.4f}")
        
        if val_metrics_dict['auc'] > best_val_auc:
            best_val_auc = val_metrics_dict['auc']
            best_model_state = model.state_dict().copy()
            trial.report(best_val_auc, epoch)  # Optuna에 보고
        
        scheduler.step()
        early_stopping(val_metrics_dict['auc'])
        if early_stopping.early_stop:
            print("\nEarly stopping triggered")
            break
            
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    
    # 모델 저장
    should_save = False
    if best_model_state is not None:
        try:
            should_save = best_val_auc > trial.study.best_value
        except ValueError:
            should_save = True

    if should_save:
        model_path = f"models/best_model_AUC_{best_val_auc:.4f}.pt"
        os.makedirs(os.path.dirname(model_path), exist_ok=True)
        torch.save({
            'model_state_dict': best_model_state,
            'val_auc': best_val_auc,
            'params': trial.params
        }, model_path)
    
    print(f"최종 검증 AUC: {best_val_auc:.4f}")
    return best_val_auc

In [ ]:
n_trials=10
study_name="health_gnn_optimization"
storage="optuna_health_gnn.db"

start_time = time.time()

study = optuna.create_study(
    study_name=study_name,
    direction="maximize",
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=10),
    sampler=optuna.samplers.TPESampler(seed=456)
)
print(f"새 스터디 '{study_name}' 생성됨")

print("Optuna 최적화 시작...")
study.optimize(objective, n_trials=n_trials, timeout=None, gc_after_trial=True)

elapsed_time = time.time() - start_time
print(f"최적화 완료! 소요 시간: {elapsed_time/60:.2f} 분")

# 결과 출력
best_trial = study.best_trial
print("\n최적 하이퍼파라미터:")
for param, value in best_trial.params.items():
    print(f"{param}: {value}")
print(f"Best Score (Validation AUC): {best_trial.value:.4f}")

# 결과 저장
result = {
    'best_params': best_trial.params,
    'best_score': best_trial.value,
    'study': study
}

with open('optuna_results_part3.pkl', 'wb') as f:
    pickle.dump(result, f)

[I 2025-05-28 07:55:31,724] A new study created in memory with name: health_gnn_optimization


새 스터디 'health_gnn_optimization' 생성됨
Optuna 최적화 시작...

평가 중: hidden_channels=44, lr=0.000031
Epoch 001: Val AUC = 0.5300
Epoch 020: Val AUC = 0.5196
Epoch 040: Val AUC = 0.7494
Epoch 060: Val AUC = 0.7685
Epoch 080: Val AUC = 0.8402
Epoch 100: Val AUC = 0.8602
Epoch 120: Val AUC = 0.8778
Epoch 140: Val AUC = 0.8819
Epoch 160: Val AUC = 0.8846
Epoch 180: Val AUC = 0.8843


[I 2025-05-28 10:39:51,568] Trial 0 finished with value: 0.8855147899271729 and parameters: {'HIDDEN_CHANNELS': 44, 'LEARNING_RATE': 3.0846106205268507e-05, 'WEIGHT_DECAY': 0.00022435190778459003, 'DROPOUT': 0.42340935553659154, 'LAMBDA_HEALTH': 0.6630655850409662}. Best is trial 0 with value: 0.8855147899271729.


Epoch 200: Val AUC = 0.8853
최종 검증 AUC: 0.8855

평가 중: hidden_channels=84, lr=0.004541
Epoch 001: Val AUC = 0.4983
Epoch 020: Val AUC = 0.9362
Epoch 040: Val AUC = 0.8294
Epoch 060: Val AUC = 0.9073


[I 2025-05-28 11:43:39,484] Trial 1 finished with value: 0.9410851263289476 and parameters: {'HIDDEN_CHANNELS': 84, 'LEARNING_RATE': 0.004540521216492651, 'WEIGHT_DECAY': 0.00018938797633002622, 'DROPOUT': 0.17244202245649765, 'LAMBDA_HEALTH': 0.23515238452855503}. Best is trial 1 with value: 0.9410851263289476.


...


평가 중: hidden_channels=78, lr=0.006717
Epoch 001: Val AUC = 0.8868
Epoch 020: Val AUC = 0.8811
Epoch 040: Val AUC = 0.8521
Epoch 060: Val AUC = 0.8535


[I 2025-05-28 18:42:54,926] Trial 8 finished with value: 0.9469600615582081 and parameters: {'HIDDEN_CHANNELS': 78, 'LEARNING_RATE': 0.006716915106186287, 'WEIGHT_DECAY': 2.8882640793546296e-06, 'DROPOUT': 0.4584902371544025, 'LAMBDA_HEALTH': 0.26023120386151893}. Best is trial 8 with value: 0.9469600615582081.



Early stopping triggered
최종 검증 AUC: 0.9470

평가 중: hidden_channels=83, lr=0.000013
Epoch 001: Val AUC = 0.1480


[I 2025-05-28 19:01:42,614] Trial 9 pruned. 


최적화 완료! 소요 시간: 666.18 분

최적 하이퍼파라미터:
HIDDEN_CHANNELS: 78
LEARNING_RATE: 0.006716915106186287
WEIGHT_DECAY: 2.8882640793546296e-06
DROPOUT: 0.4584902371544025
LAMBDA_HEALTH: 0.26023120386151893
Best Score (Validation AUC): 0.9470


In [4]:
all_trials = []
part_best_scores = {}

for i in range(1, 4):
    with open(f'optuna_results_part{i}.pkl', 'rb') as f:
        part_results = pickle.load(f)
    
    if 'study' in part_results:
        study = part_results['study']
        trials_data = []
        for trial in study.trials:
            if trial.value is not None:
                trials_data.append((trial.params, trial.value))
        all_trials.extend(trials_data)
    else:
        all_trials.extend(part_results.get('all_trials', []))
    
    part_best_scores[f'part{i}'] = part_results['best_score']
    
    print(f"PKL Part {i}/3 로드 완료 - Best AUC: {part_results['best_score']:.4f}")

unique_trials = {}
for params, score in all_trials:
    param_key = tuple(sorted(params.items()))
    if param_key not in unique_trials or unique_trials[param_key] < score:
        unique_trials[param_key] = score

unique_trials_list = [(dict(params), score) for params, score in unique_trials.items()]
valid_trials = [(params, score) for params, score in unique_trials_list if score is not None]

best_trial = max(valid_trials, key=lambda x: x[1])
best_params, best_score = best_trial

scores = [score for _, score in valid_trials]
mean_score = np.mean(scores)
std_score = np.std(scores)

# 결과 정리
final_results = {
    'best_params': best_params,
    'best_score': best_score,
    'all_trials': unique_trials_list,
    'statistics': {
        'total_trials_raw': len(all_trials),
        'unique_trials': len(unique_trials_list),
        'valid_trials': len(valid_trials),
        'mean_score': mean_score,
        'std_score': std_score,
        'min_score': min(scores),
        'max_score': max(scores)
    },
    'part_best_scores': part_best_scores
}

# 최종 결과 저장
with open('optuna_results_final.pkl', 'wb') as f:
    pickle.dump(final_results, f)

# 결과 출력
print("\n" + "="*50)
print("최종 통합 결과")
print("="*50)
print(f"원본 Trial 수: {len(all_trials)}")
print(f"중복 제거 후: {len(unique_trials_list)}")
print(f"유효 Trial 수: {len(valid_trials)}")
print(f"최고 성능: {best_score:.4f}")
print(f"평균 성능: {mean_score:.4f} ± {std_score:.4f}")

print("\n최적 하이퍼파라미터:")
for param, value in best_params.items():
    if isinstance(value, float):
        print(f"  {param}: {value:.6f}")
    else:
        print(f"  {param}: {value}")

print("\n구간별 최고 성능:")
for part, score in part_best_scores.items():
    print(f"  {part}: {score:.4f}")

print(f"\n결과가 'optuna_results_final.pkl'에 저장되었습니다.")

PKL Part 1/3 로드 완료 - Best AUC: 0.9529
PKL Part 2/3 로드 완료 - Best AUC: 0.9211
PKL Part 3/3 로드 완료 - Best AUC: 0.9470

최종 통합 결과
원본 Trial 수: 30
중복 제거 후: 30
유효 Trial 수: 30
최고 성능: 0.9529
평균 성능: 0.7732 ± 0.1718

최적 하이퍼파라미터:
  DROPOUT: 0.184936
  HIDDEN_CHANNELS: 18
  LAMBDA_HEALTH: 0.263642
  LEARNING_RATE: 0.008123
  WEIGHT_DECAY: 0.000314

구간별 최고 성능:
  part1: 0.9529
  part2: 0.9211
  part3: 0.9470

결과가 'optuna_results_final.pkl'에 저장되었습니다.


In [12]:
import pandas as pd

final_results['all_trials']

[({'DROPOUT': 0.3394633936788146,
   'HIDDEN_CHANNELS': 58,
   'LAMBDA_HEALTH': 0.24041677639819287,
   'LEARNING_RATE': 0.0071144760093434225,
   'WEIGHT_DECAY': 0.000157029708840554},
  0.8989788698866508),
 ({'DROPOUT': 0.34044600469728353,
   'HIDDEN_CHANNELS': 33,
   'LAMBDA_HEALTH': 0.737265320016441,
   'LEARNING_RATE': 1.493656855461762e-05,
   'WEIGHT_DECAY': 0.0003967605077052988},
  0.7837406363773131),
 ({'DROPOUT': 0.18493564427131048,
   'HIDDEN_CHANNELS': 18,
   'LAMBDA_HEALTH': 0.26364247048639056,
   'LEARNING_RATE': 0.008123245085588688,
   'WEIGHT_DECAY': 0.0003142880890840109},
  0.9529277407161398),
 ({'DROPOUT': 0.2727780074568463,
   'HIDDEN_CHANNELS': 36,
   'LAMBDA_HEALTH': 0.36210622617823773,
   'LEARNING_RATE': 8.17949947521167e-05,
   'WEIGHT_DECAY': 3.752055855124284e-05},
  0.8567405849412879),
 ({'DROPOUT': 0.2465447373174767,
   'HIDDEN_CHANNELS': 85,
   'LAMBDA_HEALTH': 0.5104629857953323,
   'LEARNING_RATE': 2.621087878265438e-05,
   'WEIGHT_DECAY': 7